##### basics

In [45]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from  sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

time: 1.79 ms (started: 2024-11-05 14:53:18 +00:00)


In [2]:
!pip install ipython-autotime
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.9 MB/s eta 0:00:00
time: 366 µs (started: 2024-11-05 14:08:03 +00:00)


In [3]:
df = pd.read_csv('jamb_exam_results.csv')
df.head()

,JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


time: 136 ms (started: 2024-11-05 14:08:04 +00:00)


##### Question 1

In [5]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df = df.drop('student_id',axis=1)
df.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,22,Female,Medium,Tertiary,1


time: 182 ms (started: 2024-11-05 14:08:16 +00:00)


In [6]:
df.isna().sum()

,0
jamb_score,0
study_hours_per_week,0
attendance_rate,0
teacher_quality,0
distance_to_school,0
school_type,0
school_location,0
extra_tutorials,0
access_to_learning_materials,0
parent_involvement,0


time: 57.4 ms (started: 2024-11-05 14:08:18 +00:00)


In [7]:
df = df.fillna(0)
df.isna().sum()

,0
jamb_score,0
study_hours_per_week,0
attendance_rate,0
teacher_quality,0
distance_to_school,0
school_type,0
school_location,0
extra_tutorials,0
access_to_learning_materials,0
parent_involvement,0


time: 26.9 ms (started: 2024-11-05 14:08:20 +00:00)


In [8]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=1)
y_train = df_train.jamb_score.values
y_val = df_val.jamb_score.values
del df_train['jamb_score']
del df_val['jamb_score']

time: 13.1 ms (started: 2024-11-05 14:08:22 +00:00)


In [9]:
dict_train = df_train.to_dict(orient='records')
dict_val = df_val.to_dict(orient='records')
dict_train[0]


{'study_hours_per_week': 20,
 'attendance_rate': 72,
 'teacher_quality': 3,
 'distance_to_school': 4.4,
 'school_type': 'Public',
 'school_location': 'Urban',
 'extra_tutorials': 'No',
 'access_to_learning_materials': 'Yes',
 'parent_involvement': 'Medium',
 'it_knowledge': 'Low',
 'age': 21,
 'gender': 'Female',
 'socioeconomic_status': 'Low',
 'parent_education_level': 0,
 'assignments_completed': 3}

time: 48 ms (started: 2024-11-05 14:08:28 +00:00)


In [10]:
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(dict_train)
X_val = dv.transform(dict_val)

time: 195 ms (started: 2024-11-05 14:08:32 +00:00)


In [9]:
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_val)
print(root_mean_squared_error(y_val,y_pred))

45.80335905740839


In [10]:
print(export_text(dt, feature_names=list(dv.get_feature_names_out())))

|--- study_hours_per_week <= 18.50
|   |--- value: [155.24]
|--- study_hours_per_week >  18.50
|   |--- value: [188.59]



##### Question 2

In [11]:
rf = RandomForestRegressor(n_estimators=10,random_state=1,n_jobs=1)
rf.fit(X_train,y_train)

RandomForestRegressor(n_estimators=10, n_jobs=1, random_state=1)

time: 1.22 s (started: 2024-11-05 14:08:39 +00:00)


In [12]:
y_pred = rf.predict(X_train)
print(root_mean_squared_error(y_train,y_pred))

17.78903210782794


In [13]:
y_pred = rf.predict(X_val)
print(root_mean_squared_error(y_val,y_pred))

42.13724207871227


##### Question 3

In [12]:
scores = []
for est in np.arange(10,200,10):
    rf = RandomForestRegressor(n_estimators=est,random_state=1,n_jobs=1)
    rf.fit(X_train,y_train)
    y_pred = rf.predict(X_val)
    scores.append({
        'est' : est,
        'RMSE' : round(root_mean_squared_error(y_val,y_pred),3)
    })

time: 3min 26s (started: 2024-11-05 14:09:06 +00:00)


In [15]:
df_scores = pd.DataFrame(data=scores)
df_scores

,est,RMSE
0,10,42.137
1,20,41.461
2,30,41.106
3,40,40.917
4,50,40.852
5,60,40.784
6,70,40.677
7,80,40.539
8,90,40.504
9,100,40.517


##### Question 4

In [13]:
max_depth = [10, 15, 20, 25]
scores = []

for d in max_depth:
    print('***DEPTH***',d)
    for est in np.arange(10,200,10):
        print('***EST***',est)
        rf = RandomForestRegressor(n_estimators=est,random_state=1,n_jobs=1,max_depth=d)
        rf.fit(X_train,y_train)
        y_pred = rf.predict(X_val)
        scores.append({
            'depth' : d,
            'est' : est,
            'RMSE' : round(root_mean_squared_error(y_val,y_pred),3)
        })


***DEPTH*** 10
***EST*** 10
***EST*** 20
***EST*** 30
***EST*** 40
***EST*** 50
***EST*** 60
***EST*** 70
***EST*** 80
***EST*** 90
***EST*** 100
***EST*** 110
***EST*** 120
***EST*** 130
***EST*** 140
***EST*** 150
***EST*** 160
***EST*** 170
***EST*** 180
***EST*** 190
***DEPTH*** 15
***EST*** 10
***EST*** 20
***EST*** 30
***EST*** 40
***EST*** 50
***EST*** 60
***EST*** 70
***EST*** 80
***EST*** 90
***EST*** 100
***EST*** 110
***EST*** 120
***EST*** 130
***EST*** 140
***EST*** 150
***EST*** 160
***EST*** 170
***EST*** 180
***EST*** 190
***DEPTH*** 20
***EST*** 10
***EST*** 20
***EST*** 30
***EST*** 40
***EST*** 50
***EST*** 60
***EST*** 70
***EST*** 80
***EST*** 90
***EST*** 100
***EST*** 110
***EST*** 120
***EST*** 130
***EST*** 140
***EST*** 150
***EST*** 160
***EST*** 170
***EST*** 180
***EST*** 190
***DEPTH*** 25
***EST*** 10
***EST*** 20
***EST*** 30
***EST*** 40
***EST*** 50
***EST*** 60
***EST*** 70
***EST*** 80
***EST*** 90
***EST*** 100
***EST*** 110
***EST*** 120
***EST*** 

In [15]:
df_scores = pd.DataFrame(data=scores)
df_scores = df_scores.sort_values(by='RMSE')
df_scores.head(1)

,depth,est,RMSE
8,10,90,40.174


time: 14 ms (started: 2024-11-05 14:24:18 +00:00)


##### Question 5

In [16]:
rf = RandomForestRegressor(n_estimators=10,max_depth=20,random_state=1,n_jobs=1)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_val)

time: 860 ms (started: 2024-11-05 14:30:19 +00:00)


In [29]:
importances = pd.Series(rf.feature_importances_,index=dv.get_feature_names_out())

time: 4.03 ms (started: 2024-11-05 14:44:47 +00:00)


In [32]:
df_importances = pd.DataFrame(importances)
df_importances.columns = ['importance']
df_importances = df_importances.sort_values(by='importance',ascending=False)


time: 1.83 ms (started: 2024-11-05 14:45:15 +00:00)


In [33]:
df_importances.head(2)

,importance
study_hours_per_week,0.248354
attendance_rate,0.149729


time: 8.6 ms (started: 2024-11-05 14:45:18 +00:00)


##### Question 6

In [34]:
!pip install xgboost

time: 2.92 s (started: 2024-11-05 14:47:27 +00:00)


In [35]:
import xgboost as xgb

time: 346 ms (started: 2024-11-05 14:47:43 +00:00)


In [36]:
features = list(dv.get_feature_names_out())
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

time: 158 ms (started: 2024-11-05 14:50:23 +00:00)


In [105]:
%%capture output
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dval, num_boost_round=100,
                  verbose_eval=5)

time: 2.1 s (started: 2024-11-05 15:30:41 +00:00)


In [106]:
y_pred = model.predict(dval)
round(root_mean_squared_error(y_val, y_pred),3)

1.669

time: 8.46 ms (started: 2024-11-05 15:30:49 +00:00)


In [107]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dval, num_boost_round=100,
                  verbose_eval=5)

time: 532 ms (started: 2024-11-05 15:31:00 +00:00)


In [108]:
y_pred = model.predict(dval)
round(root_mean_squared_error(y_val, y_pred),3)

11.968

time: 14.5 ms (started: 2024-11-05 15:31:09 +00:00)
